In [1]:
# Install required packages

%pip install -U langgraph langchain_community langchain_openai langsmith langgraph-swarm 

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 3.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 5.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/12.6 MB 7.2 MB/s eta 0:00:02
   --------- ------------------------------ 3.1/12.6 MB 8.4 MB/s eta 0:00:02
   ---------------- ----------------------- 5.2/12.6 MB 8.9 MB/s eta 0:00:01
   ------------------------ --------------- 7.6/12.6 MB 9.2 MB/s eta 0:00:01
   ------------------------------ --------- 9.7/12.6 MB 9.5 MB/s eta 0:00:01
   -------------------------------------- - 12.1/12.6 MB 9.6 MB/s eta 0:00:01
   ----------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
# Environment Variable Initialization

import getpass
import os

def _set_if_undefined(var_name: str):
    """
    Set an environment variable if it is not already defined.
    
    Args:
        var_name (str): Name of the environment variable to set.
    """
    if not os.environ.get(var_name):
        # Securely prompt the user for input without echoing it on screen
        os.environ[var_name] = getpass.getpass(f"Please provide your {var_name}: ")

# ---- Environment Variables Required ----


_set_if_undefined("OPENAI_API_KEY")         # API key for OpenAI models
_set_if_undefined("ANTHROPIC_API_KEY")      # API key for OpenAI models
_set_if_undefined("LANGSMITH_TRACING")      # Enable LangSmith tracing ("true" to enable)
_set_if_undefined("LANGSMITH_API_KEY")      # API key for LangSmith platform
_set_if_undefined("OPENAI_MODEL")           # Model name (e.g., "gpt-4.1" "gpt-4o", "gpt-3.5-turbo")
_set_if_undefined("ANTHROPIC_MODEL")        # Model name (e.g., "claude-3-7-sonnet-latest")



In [31]:
# Network
# Each agent can communicate with every other agent (many-to-many connections) 
# and can decide which agent to call next.

import sys
import os
from typing import Literal
from langchain_core.messages import HumanMessage
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langgraph.types import Command
from langgraph.graph import StateGraph, MessagesState, START, END

# Load the model name from environment variables
openai_model = os.environ["OPENAI_MODEL"]
# Initialize the LLM (Large Language Model) interface
openai_llm = ChatOpenAI(model=openai_model)


class Router(TypedDict):
    """Next coaches to apply brakes. If all coaches are already braking, route to __end__."""
    next_coaches: list
   
class CoachStatus(TypedDict):
    """coach braking status. If already braking return True otherwise False."""
    braking: Literal[True, False]

system_prompt = (
   "You are part of a train with multiple coaches (coach_1, coach_2, coach_3, coach_4, coach_5). "
   "Each coach can apply brakes upon a user command. "
   "All coaches must apply their brakes simultaneously, "
   "and each coach should update its status when doing so. "
   "Given the following user request, "
   "respond with the name of coaches that has not yet applied the brakes and should act next. "
   f"If all coaches have already applied the brakes, respond with {END}. "
)

def coach_1(state: MessagesState) -> Command[Literal["coach_2", "coach_3","coach_4","coach_5", END]]:
    this_coach_name = sys._getframe(  ).f_code.co_name
    return coach_logic(this_coach_name=this_coach_name, state=state)

def coach_2(state: MessagesState) -> Command[Literal["coach_1", "coach_3","coach_4","coach_5", END]]:
    this_coach_name = sys._getframe(  ).f_code.co_name
    return coach_logic(this_coach_name=this_coach_name, state=state)

def coach_3(state: MessagesState) -> Command[Literal["coach_1", "coach_2","coach_4","coach_5", END]]:
    this_coach_name = sys._getframe(  ).f_code.co_name
    return coach_logic(this_coach_name=this_coach_name, state=state)

def coach_4(state: MessagesState) -> Command[Literal["coach_1", "coach_2","coach_3","coach_5", END]]:
    this_coach_name = sys._getframe(  ).f_code.co_name
    return coach_logic(this_coach_name=this_coach_name, state=state)

def coach_5(state: MessagesState) -> Command[Literal["coach_1", "coach_2","coach_3","coach_4", END]]:
    this_coach_name = sys._getframe(  ).f_code.co_name
    return coach_logic(this_coach_name=this_coach_name, state=state)

def coach_logic(this_coach_name: str, state: MessagesState) -> Command[Literal["coach_1", "coach_2", "coach_3","coach_4","coach_5", END]]:
    # you can pass relevant parts of the state to the LLM (e.g., state["messages"])
    # to determine which agent to call next. a common pattern is to call the model
    # with a structured output (e.g. force it to return an output with a "next_coach" field)
    messages = [
        {"role": "system", "content": system_prompt},
    ] + state["messages"] 
            
    response = openai_llm.with_structured_output(Router).invoke(messages)

    # Filter out self from the list
    recipients = [coach for coach in response["next_coaches"] if coach != this_coach_name]

    # check if already initiated brake.
    messages = state["messages"] + [ f"Looking at the messages history, identify if {this_coach_name} is already braking" ]

    coach_staus = openai_llm.with_structured_output(CoachStatus).invoke(messages)

    updated_message = "I am already Braking" if coach_staus["braking"] == True else "I am Braking"

    # route to one of the agents or exit based on the LLM's decision
    # if the LLM returns "__end__", the graph will finish execution
    return Command(
        goto=recipients,
        update={
            "messages": [
                HumanMessage(content=updated_message, name=this_coach_name)
            ]
        },
    )




In [32]:
# ---- Define network graph ----

builder = StateGraph(MessagesState)
builder.add_node(coach_1)
builder.add_node(coach_2)
builder.add_node(coach_3)
builder.add_node(coach_4)
builder.add_node(coach_5)

builder.add_edge(START, "coach_1")
network = builder.compile()


In [33]:
# ---- Stream user input ----

for s in network.stream(
    {"messages": [("user", "Brake")]}, debug=True):
    print(s)
    print("============================")

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'Brake')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'Brake')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='Brake', additional_kwargs={}, response_metadata={}, id='5f27c8ea-8ecc-4a7b-b5c1-cff9c784929e')]}
[1:tasks] Starting 1 task for step 1:
- coach_1 -> {'messages': [HumanMessage(content='Brake', additional_kwargs={}, response_metadata={}, id='5f27c8ea-8ecc-4a7b-b5c1-cff9c784929e')]}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [HumanMessage(content='I am Braking', additional_kwargs={}, response_metadata={}, name='coach_1')]
[1:checkpoint] State at the end of step 1:
{'messages': [HumanMessage(content='Brake', additional_kwargs={}, response_metadata={}, id='5f27c8ea-8ecc-4a7b-b5c1-cff9c784929e'),
              HumanMessage(content='I am Braking', additional_k